<a href="https://colab.research.google.com/github/JavIonBek/chatbot-uz/blob/main/chatbot2-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot model
Creating a chatbot model

In [ ]:
# to connect to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Gathering data
Let's import some libraries and load data to get started!

In [ ]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import pickle
import random

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import tensorflow as tf
import keras
print(keras.__version__)
print(tf.__version__)
print(nltk.__version__)

2.4.3
2.4.1
3.2.5


In [ ]:
# import our chat-bot intents file
import json
with open('/content/drive/MyDrive/Colab Notebooks/chatbot-uz/data/intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), 'documents')
# classes = intents
print(len(classes), 'classes', classes)
# words = all words, vocabulary
print(len(words), 'unique stemmed words', words)

40 documents
6 classes ['kim_san', 'qonun_buzarlikni_yuklash', 'rahmat', 'rezyume_yuborish', 'salom', 'xayr']
66 unique stemmed words ["'", 'aleyk', 'assalom', 'assalomu', 'aссалом', 'aссалому', "bo'ladi", "bo'ling", 'buzarlik', 'haqid', 'hayrl', 'ism', 'kattakon', 'kim', "ko'rishguncha", 'kun', "ma'lumot", 'nim', 'qalays', 'qanday', 'qilib', 'qilsam', 'qonun', 'rahm', 'raxm', 'rezyum', 'salom', 'san', 'saytd', 'sog', 'uchun', 'xab', 'xayr', 'yordam', 'yub', 'yuborsam', 'yuklasam', 'алейкум', 'бузарлик', 'бўлади', 'бўлинг', 'исминг', 'каттакон', 'ким', 'кўришгунча', 'маълумот', 'нима', 'рахмат', 'раҳмат', 'резюме', 'сайтдан', 'салом', 'сан', 'соғ', 'учун', 'хабар', 'хайр', 'юбориш', 'юборсам', 'юкласам', 'ёрдамингиз', 'қандай', 'қилиб', 'қилсам', 'қонун', 'ҳақида']


In [ ]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

## Choose a model

### To save the trained keras model in a pickle file
https://github.com/tensorflow/tensorflow/issues/34697#issuecomment-627193883

In [ ]:
# To save the trained keras model in a pickle file

from tensorflow.keras.models import Model
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils


def unpack(model, training_config, weights):
  restored_model = deserialize(model)
  if training_config is not None:
    restored_model.compile(
      **saving_utils.compile_args_from_training_config(
          training_config
      )
    )
  restored_model.set_weights(weights)
  return restored_model

# Hotfix function
def make_keras_picklable():

  def __reduce__(self):
    model_metadata = saving_utils.model_metadata(self)
    training_config = model_metadata.get('training_config', None)
    model = serialize(self)
    weights = self.get_weights()
    return (unpack, (model, training_config, weights))

  cls = Model
  cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))  # None not
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [ ]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8576      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 17,222
Trainable params: 17,222
Non-trainable params: 0
_________________________________________________________________


## Training

In [ ]:
# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
8/8 [==============================] - 3s 3ms/step - loss: 1.8039 - accuracy: 0.1852
Epoch 2/200
8/8 [==============================] - 0s 3ms/step - loss: 1.7814 - accuracy: 0.2587
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 1.7288 - accuracy: 0.3471
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6116 - accuracy: 0.3950
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 1.6449 - accuracy: 0.3878
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 1.5665 - accuracy: 0.4465
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 1.4514 - accuracy: 0.6899
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 1.4138 - accuracy: 0.6515
Epoch 9/200
8/8 [==============================] - 0s 3ms/step - loss: 1.3151 - accuracy: 0.5616
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.3627 - accuracy: 0.5516
Epoch 11/200
8/8 [===========

## Testing

In [ ]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ('found in bag: %s' % w)

    return(np.array(bag))

In [ ]:
p = bow('Assalomu aleykum, yaxshimisiz', words)
print(p)
print(classes)

found in bag: assalomu
found in bag: aleyk
[0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['kim_san', 'qonun_buzarlikni_yuklash', 'rahmat', 'rezyume_yuborish', 'salom', 'xayr']


In [ ]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(model.predict(inputvar))

[[2.5724632e-06 2.4721710e-06 9.0833882e-06 1.0854361e-05 9.9996459e-01
  1.0412777e-05]]


## Save model

In [ ]:
# Save model
with open('/content/drive/MyDrive/Colab Notebooks/chatbot-uz/model/chatbot-model.pkl', 'wb') as f:
  pickle.dump(model, f)

In [ ]:
# save all of our data structures
pickle.dump({'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open('/content/drive/MyDrive/Colab Notebooks/chatbot-uz/model/chatbot-data.pkl', 'wb'))